In [20]:
import os
import pandas as pd
import numpy as np
import plotly.express as px

import nbformat


os.getcwd()

'c:\\Users\\PLHT09191\\dev\\viz_velos\\src'

In [21]:

dtypes = {"X":float, "Y":float, 
    "capacite":str,
    "couverture":bool, "surveillance":bool,
    "lumiere":str, "proprietaire":str, "gestionnaire":str
}
data_folder = "../data/"
df = pd.read_csv(data_folder + "data.csv", sep=";", usecols=dtypes.keys(), dtype=dtypes, on_bad_lines="skip")
df["capacite"] = pd.to_numeric(df["capacite"], errors="coerce")
df["lumiere"] = df["lumiere"].replace({"true":True, "false":False}).astype("boolean")
df.to_csv(data_folder + "clean_data.csv", index=False)

In [22]:
capas = df.loc[:, ["gestionnaire", "capacite"]]
capas = capas.dropna()
capas = (capas
        .groupby("gestionnaire")
        .agg(c=("capacite", "count"), capacite_totale=("capacite", "sum"))
        .reset_index()
        .sort_values("capacite_totale", ascending=False)
)
print(capas.quantile())
capas = capas[capas.c > 1]
capas

c                   1.0
capacite_totale    14.5
Name: 0.5, dtype: float64


,gestionnaire,c,capacite_totale
367,Grand Lyon,2011,22357.0
739,SNCF,225,5809.0
791,TER Grand-Est,105,3744.0
670,Pau,173,1736.0
752,STIF,24,1632.0
...,...,...,...
570,Mairie de Pocé-sur-Cisse,2,4.0
940,Ville de St Cyr sur Loire,2,4.0
931,Ville de Redon,2,4.0
643,Nantes métropole,2,3.0


In [23]:

fig = px.bar(capas[capas["gestionnaire"].str.contains("a")][:50], x="gestionnaire", y="capacite_totale")
fig.show()

In [24]:
capacites = (df[(df["gestionnaire"].isna()) | (df["gestionnaire"].str.lower().str.contains("sncf"))]
            .dropna(subset=["capacite"])
            .sort_values("capacite")[:100]
)
fig = px.scatter_geo(capacites["capacite"], lat=capacites.Y, lon=capacites.X, scope="europe")
fig.show()

In [49]:
data = df
w_couv = 1
w_surv = 1
w_lum = 1
surete = data.loc[:, ["gestionnaire", "couverture", "surveillance"]].dropna()
surete = surete.groupby("gestionnaire")[["couverture", "surveillance"]].sum().reset_index()
surete["total"] = (
    w_couv * surete["couverture"]
    + w_surv * surete["surveillance"]
    # + w_lum * surete["lumiere"]
)
surete = surete.sort_values("total", ascending=False)[:10]
fig = px.bar(surete, x=surete["gestionnaire"], y=surete["total"])
